# Análisis de Importación de Vehículos desde el Portal SAT

In [9]:
import os
import re
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import zipfile

In [10]:
urlSat = "https://portal.sat.gob.gt/portal/alza-e-importacion-vehiculos/#1510763502681-dff4b62b-fd76"
baseUrl = "https://portal.sat.gob.gt"
rutaSalida = "./datos/enlaces-importacion-2024_2025.txt"
carpetaZips = "./datos/zips/"
carpetaDescomprimidos = "./datos/descomprimidos/"

## Código para bajar los enlaces necesarios

Podemos en un futuro solo aplicar el codigo para descarga de descomprimir los archivos dado los enlaces sin hacer la búsqueda.

In [5]:
def obtenerImportacionesZips(url):
    resp = requests.get(url, timeout=15)
    resp.raise_for_status()

    soup = BeautifulSoup(resp.text, "html.parser")
    patronZip = re.compile(r"\.zip$", re.I)
    patronImportacion = re.compile(r"/importacion-de-vehiculos/", re.I)

    enlaces = set()

    for a in soup.find_all("a", href=True):
        href = a["href"]
        if (
            patronZip.search(href)
            and ("2024" in href or "2025" in href)
            and patronImportacion.search(href)
        ):
            enlaces.add(urljoin(baseUrl, href))

    return sorted(enlaces)

In [6]:
def guardarEnlacesImportacion(forzar=False):
    if os.path.exists(rutaSalida) and not forzar:
        respuesta = input(f"El archivo '{rutaSalida}' ya existe. ¿Deseas rehacer el proceso? (s/n): ").strip().lower()
        if respuesta != "s":
            print("Proceso cancelado.")
            return

    print("Obteniendo enlaces...")
    enlaces = obtenerImportacionesZips(urlSat)

    os.makedirs(os.path.dirname(rutaSalida), exist_ok=True)

    with open(rutaSalida, "w", encoding="utf-8") as f:
        f.write("\n".join(enlaces))

    print(f"{len(enlaces)} enlaces guardados en {rutaSalida}")

### Hacer WebScrapping

Se puede cambiar el valor **forzar** a true para hacer la descarga automatica de los enlaces (aún si existe el archivo de enlaces en `./data/enlaces-importacion-2024_2025.txt`)

In [7]:
guardarEnlacesImportacion(forzar=False)

Obteniendo enlaces...
17 enlaces guardados en ./datos/enlaces-importacion-2024_2025.txt


## Código para descomprimir los archivos Zip

In [12]:
def descargarYDescomprimirZips(ejecutar=True):
    """Descarga y descomprime los ZIP listados en rutaSalida si ejecutar=True."""
    if not ejecutar:
        print("Descarga y descompresión omitidas (ejecutar=False).")
        return

    if not os.path.exists(rutaSalida):
        print(f"No existe el archivo de enlaces: {rutaSalida}")
        return

    os.makedirs(carpetaZips, exist_ok=True)
    os.makedirs(carpetaDescomprimidos, exist_ok=True)

    with open(rutaSalida, "r", encoding="utf-8") as f:
        enlaces = [line.strip() for line in f if line.strip()]

    for enlace in enlaces:
        nombreZip = os.path.basename(enlace)
        rutaZip = os.path.join(carpetaZips, nombreZip)

        if not os.path.exists(rutaZip):
            print(f"Descargando: {nombreZip}...")
            try:
                r = requests.get(enlace, timeout=30)
                r.raise_for_status()
                with open(rutaZip, "wb") as fzip:
                    fzip.write(r.content)
                print(f"Descargado: {rutaZip}")
            except Exception as e:
                print(f"Error al descargar {enlace}: {e}")
                continue
        else:
            print(f"Ya existe: {rutaZip}")

        try:
            with zipfile.ZipFile(rutaZip, 'r') as zip_ref:
                zip_ref.extractall(carpetaDescomprimidos)
                print(f"Descomprimido en: {carpetaDescomprimidos}")
        except zipfile.BadZipFile:
            print(f"Archivo ZIP corrupto o inválido: {rutaZip}")

### Descomprimir Zips

Se puede modificar el campo `ejecutar` a true o false para ejecutar el proceso de descarga y descomprimir archivos.

In [13]:
descargarYDescomprimirZips(ejecutar=True)

Descargando: importacion_de_vehiculos_2024_enero.zip...
Descargado: ./datos/zips/importacion_de_vehiculos_2024_enero.zip
Descomprimido en: ./datos/descomprimidos/
Descargando: importacion_de_vehiculos_2024_febrero.zip...
Descargado: ./datos/zips/importacion_de_vehiculos_2024_febrero.zip
Descomprimido en: ./datos/descomprimidos/
Descargando: importacion_de_vehiculos_2024_marzo.zip...
Descargado: ./datos/zips/importacion_de_vehiculos_2024_marzo.zip
Descomprimido en: ./datos/descomprimidos/
Descargando: importacion_de_vehiculos_2024_abril.zip...
Descargado: ./datos/zips/importacion_de_vehiculos_2024_abril.zip
Descomprimido en: ./datos/descomprimidos/
Descargando: importacion_de_vehiculos_2024_mayo.zip...
Descargado: ./datos/zips/importacion_de_vehiculos_2024_mayo.zip
Descomprimido en: ./datos/descomprimidos/
Descargando: importacion_de_vehiculos_2024_junio.zip...
Descargado: ./datos/zips/importacion_de_vehiculos_2024_junio.zip
Descomprimido en: ./datos/descomprimidos/
Descargando: importa